# Model Reviewer and Building GeoSentences
A Model Reviewer that produces static images of generated models from a model generator function. A function that generates a history is passed and the Model Reviewer will render, display, and allow the option to reject or save the model.

In [1]:
import pyvista as pv
pv.set_jupyter_backend('static')

## Model History Generation
The generation package contains a set of GeoWords and geological sentence builder functions that allow for specifying mini-histories formed of one or more GeoProcesses that have generating random variables attached to them. A `GeoWord` object is an object that packages one or more `GeoProcess` with constrained random variable parameters. It offers a convenient way to name parameterized snippets of geological history.

The atomic operations on the model are the `GeoProcess` depositions and transformations that function as leaf nodes in a computational tree. These are like the basic building blocks or 'alphabet' of the model that can be used to form words. 

<img src="https://i.imgur.com/wudCw3Q.png" width="800" >

Geowords are an extension of the tree computation that fills in the missing parameters of the `GeoProcess` leaves and wraps the snippets in a `CompoundProcess`

Note that we can view a table of the history in Jupyter by simply calling the `display(model)` function. This will render the history in a human-readable html table format.

In [2]:
import structgeo.generation as gen
from structgeo.model import GeoModel
# Geo sentence fromed of geowords
sentence = [gen.InfiniteBasement() , gen.CoarseRepeatSediment(), gen.FourierFold(), gen.SingleRandSediment(), gen.FineRepeatSediment()]
hist = gen.generate_history(sentence) 

# Add history to a model to generate a 3D model
model = GeoModel(bounds = (-1000,1000), resolution = 128)
model.add_history(hist)
model.compute_model(normalize = True)
display(model) # Display a table of model information

GeoModel(name=model, bounds=((-1000, 1000), (-1000, 1000), (-1000, 1000)), resolution=(128, 128, 128))

##

### Model Review

The model review is a simple Jupyter Notebook interface with buttons that allows for the review of the output of a defined model generator function. Using the `GeoWord` context


##
Model visualizations are available in the plot package. The 

In [3]:
from structgeo.plot import ModelReviewer

# Save directory for models
DEFAULT_BASE_DIR = "../saved_models"

# Model resolution and bounds
res = (128,128,64)
bounds = ((-3840,3840),(-3840,3840),(-1920,1920)) 
def generate_model():
    # Generate a randomized history from geowords, one single sample
    hist = gen.generate_history(sentence)    
    # Generate a model
    model = GeoModel(bounds = bounds, resolution = res)
    model.add_history(hist)
    model.compute_model(normalize = True)
    return model

reviewer = ModelReviewer(generate_model_func=generate_model, base_dir=DEFAULT_BASE_DIR, show_history=True, single_view=False)
reviewer.start_review()

Output()

## Generation via Markov Chain

Models can also be generated using a Markov sampling scheme on general GeoWord categories. This is the same type of sampling regime that is used to generate infinite synthetic datasets for Torch dataloaders. 

The `MarkovGeostoryGenerator` class reads a Markov matrix from a CSV file that specifies the transition probabilities between different geological events. The details of the file formatting can be found in the `generation/model_generators.py` MarkovMatrixParser class which handles the conversion into a MarkovChain object, supported by the library `PyDTMC` https://github.com/TommasoBelluzzo/PyDTMC. The file can be edited in a spreadsheet and exported as CSV from: https://docs.google.com/spreadsheets/d/1OzP1ewVcsB4IKpeLPMQyVwLWbeFcTm4OtPxi-n7J5Ng/edit?gid=0#gid=0

A configuration file is provided by default if no configuration path is specified: i.e. `config = None`

Note that the histories that are generated using this process can be quite long and complex

In [5]:
from structgeo.plot import ModelReviewer
from structgeo.generation import MarkovGeostoryGenerator

model_generator = MarkovGeostoryGenerator(
    model_bounds=((-3840, 3840), (-3840, 3840), (-1920, 1920)),
    model_resolution=(128, 128, 64),
    config=None,
)
reviewer = ModelReviewer(
    generate_model_func=model_generator.generate_model, base_dir=DEFAULT_BASE_DIR, show_history=True, single_view=False
)
reviewer.start_review()

c:\users\sghys\summer2024\structuralgeo\src\structgeo\generation\categorical_events.py:112: RuntimeWarning: Probabilities sum to 0.8000, but should sum to 1.0. Renormalizing.
  warnings.warn(


Output()

## Forming New GeoWords

The contract for a geoword is that it simply needs to have a `build_ history()` function that will populate the `self.history` field.  For example we want to pair a fault with a dike randomly placed in the model. We can create a new `GeoWord` object that will encapsulate the random variables and the underlying processes. 

In [10]:
from structgeo.model import Fault, DikePlane
from structgeo.probability import random_point_in_ellipsoid
import numpy as np

class MyFaultDikeWord(gen._GeoWord):
    def build_history(self):
        bounds = ((-3840,3840),(-3840,3840),(-1920,1920)) 
        strike = np.random.uniform(0, 360)
        dip = np.random.uniform(75,90)
        rake = np.random.uniform(0,360)
        origin = random_point_in_ellipsoid(bounds)
        # First a Fault
        fault_params = {
            'strike' : strike,
            'dip' : dip,
            'rake' : rake,
            'amplitude' : np.random.lognormal(0,.25)*200,           
            'origin' :    origin
        }
        fault = Fault(**fault_params)
        # Then a Dike in the same area
        dike_params = {
            'strike' : strike,
            'dip' : dip,          
            'width' : np.random.normal(150,50),
            'origin' :    origin
        }
        dike = DikePlane(**dike_params)
        
        self.add_process(fault)
        self.add_process(dike)
        # No return value needed
        

Now adding a new `GeoWord` object that encapsulates the fault and dike processes. To a sentence:

In [11]:
sentence = [gen.InfiniteBasement() , gen.CoarseRepeatSediment(), gen.FourierFold(), gen.SingleRandSediment(), gen.FineRepeatSediment(), MyFaultDikeWord()]

# Save directory for models
DEFAULT_BASE_DIR = "../saved_models"

# Model resolution and bounds
res = (256,256,128)
bounds = ((-3840,3840),(-3840,3840),(-1920,1920)) 
def generate_model():
    # Generate a randomized history from geowords, one single sample
    hist = gen.generate_history(sentence)    
    # Generate a model
    model = GeoModel(bounds = bounds, resolution = res)
    model.add_history(hist)
    model.compute_model(normalize = True)
    return model

reviewer = ModelReviewer(generate_model_func=generate_model, base_dir=DEFAULT_BASE_DIR)
reviewer.start_review()

Output()

Review exited.


### Nested GeoWords
Similar to the underlying GeoProcesses, GeoWords can be nested to form more complex histories. For example, we can create a `GeoWord` that encapsulates a fault and a dike, and then use that in another `GeoWord` that encapsulates a basin and the fault-dike. A word can consist one or more GeoWords and GeoProcesses or a combination of both.

Have a look at the available GeoWords and GeoProcesses to work with for inspiration.

In [12]:
class KitchenSink(gen._GeoWord):
    def build_history(self):
        self.add_process(gen.InfiniteBasement())
        self.add_process(gen.CoarseRepeatSediment())
        # Sample randomly from a list of geowords
        word_options = [gen.CoarseRepeatSediment, gen.FourierFold, gen.ShapedFold, gen.SingleRandSediment, gen.FineRepeatSediment, MyFaultDikeWord]
        for i in range(np.random.randint(2,5)):
            word = np.random.choice(word_options)
            self.add_process(word())
        # No return value needed       

In [13]:
sentence = [KitchenSink(), KitchenSink(), KitchenSink(),KitchenSink()]
# Save directory for models
DEFAULT_BASE_DIR = "../saved_models"

# Model resolution and bounds
res = (256,256,128)
bounds = ((-3840,3840),(-3840,3840),(-1920,1920)) 
def generate_model():
    # Generate a randomized history from geowords, one single sample
    hist = gen.generate_history(sentence)    
    # Generate a model
    model = GeoModel(bounds = bounds, resolution = res)
    model.add_history(hist)
    model.compute_model(normalize = True)
    return model

reviewer = ModelReviewer(generate_model_func=generate_model, base_dir=DEFAULT_BASE_DIR)
reviewer.start_review()

Output()

In [ ]:
import structgeo.plot as geovis
model = generate_model()
p = geovis.volview(model,)